In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from bert.tokenization.bert_tokenization import FullTokenizer

2022-04-27 02:15:29.192857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64
2022-04-27 02:15:29.192923: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.5.0


In [2]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)

2022-04-27 02:16:35.892125: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-04-27 02:16:35.961881: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 02:16:35.962385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2022-04-27 02:16:35.962442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 02:16:35.963147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID

In [13]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [19]:
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

In [14]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        #Cutting down the excess length
        tokens = tokens[0:max_seq_length]
        return [1]*len(tokens)
    else:
        return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        #Cutting down the excess length
        tokens = tokens[:max_seq_length]
        segments = []
        current_segment_id = 0
        for token in tokens:
            segments.append(current_segment_id)
            if token == "[SEP]":
                current_segment_id = 1
        return segments
    else:
        segments = []
        current_segment_id = 0
        for token in tokens:
            segments.append(current_segment_id)
            if token == "[SEP]":
                current_segment_id = 1
        return segments + [0] * (max_seq_length - len(tokens))

def get_ids(tokens, tokenizer, max_seq_length):    
    if len(tokens)>max_seq_length:
        tokens = tokens[:max_seq_length]
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        return token_ids
    else:
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
        return input_ids

In [15]:
def prep(s, get = 'id'):
    stokens = tokenizer.tokenize(s)
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    if get == 'id':
        input_ids = get_ids(stokens, tokenizer, max_seq_length)
        return input_ids
    elif get == 'mask':
        input_masks = get_masks(stokens, max_seq_length)
        return input_masks
    else:
        input_segments = get_segments(stokens, max_seq_length)
        return input_segments

In [16]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [17]:
#TESTING
max_seq_length = 128  #This number will determine the number of tokens
 
#An example for tokenization
s1 = "But the most painful was the huge reversal in fee income, unheard of among private sector lenders. Essentially, it means that Yes Bank took it for granted that fees on structured loan deals will be paid and accounted for upfront on its books. As borrowers turned defaulters, the fees tied to these loan deals fell off the cracks. Gill has now vowed to shift to a safer accounting practice of amortizing fee income rather than booking these upfront. Gill’s move to mend past ways means that there will be no nasty surprises in the future. This is good news considering that investors love a clean image and loathe uncertainties.But there is no gain without pain and the promise of a strong and stable balance sheet comes with some sacrifices as well. Investors will have to give up the hopes of phenomenal growth, a promise made by Kapoor."
#s1 = train['STORY'].iloc[0]
stokens1 = tokenizer.tokenize(s1)
stokens1 = ["[CLS]"] + stokens1 + ["[SEP]"]
 
input_ids1 = get_ids(stokens1, tokenizer, max_seq_length)
input_masks1 = get_masks(stokens1, max_seq_length)
input_segments1 = get_segments(stokens1, max_seq_length)
 
print("IDS # len:" , len(input_ids1), " ::: ",input_ids1)
print("MASKS # len:" , len(input_masks1), " ::: ",input_masks1)
print("SEGEMNTS # len:" , len(input_segments1), " ::: ",input_segments1)

IDS # len: 128  :::  [101, 2021, 1996, 2087, 9145, 2001, 1996, 4121, 23163, 1999, 7408, 3318, 1010, 4895, 26362, 1997, 2426, 2797, 4753, 18496, 2545, 1012, 7687, 1010, 2009, 2965, 2008, 2748, 2924, 2165, 2009, 2005, 4379, 2008, 9883, 2006, 14336, 5414, 9144, 2097, 2022, 3825, 1998, 14729, 2005, 2039, 12792, 2006, 2049, 2808, 1012, 2004, 17781, 2545, 2357, 12398, 2545, 1010, 1996, 9883, 5079, 2000, 2122, 5414, 9144, 3062, 2125, 1996, 15288, 1012, 12267, 2038, 2085, 18152, 2000, 5670, 2000, 1037, 13726, 9529, 3218, 1997, 16095, 3775, 6774, 7408, 3318, 2738, 2084, 21725, 2122, 2039, 12792, 1012, 12267, 1521, 1055, 2693, 2000, 2273, 2094, 2627, 3971, 2965, 2008, 2045, 2097, 2022, 2053, 11808, 20096, 1999, 1996, 2925, 1012, 2023, 2003, 2204, 2739, 6195, 2008, 9387, 2293, 1037, 4550, 3746, 1998, 8840]
MASKS # len: 128  :::  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [36]:
input_ids1_tf = tf.convert_to_tensor(input_ids1)
input_masks1_tf = tf.convert_to_tensor(input_masks1)
input_segments1_tf = tf.convert_to_tensor(input_segments1)

In [37]:
pool_embs, all_embs = model.predict([input_ids1_tf, input_masks1_tf, input_segments1_tf])

In [43]:
pool_embs


array([[-0.7348337 ,  0.12700568,  0.5870974 , ...,  0.54593676,
        -0.47063637,  0.80095446],
       [-0.30164298,  0.02687909,  0.67716587, ...,  0.6470015 ,
        -0.5090403 ,  0.48810825],
       [-0.38624573,  0.20944679,  0.57931405, ...,  0.75650096,
        -0.28002295,  0.33652538],
       ...,
       [-0.13146849, -0.22396435,  0.21836293, ..., -0.6283861 ,
         0.08465796,  0.28880864],
       [-0.19912848,  0.24072008, -0.37078485, ..., -0.57031006,
        -0.13679793,  0.7908602 ],
       [-0.34718457, -0.09375188,  0.69986916, ...,  0.5418732 ,
        -0.5776768 ,  0.79960287]], dtype=float32)

In [41]:
all_embs.shape

(128, 1, 768)

In [42]:
stokens1

['[CLS]',
 'but',
 'the',
 'most',
 'painful',
 'was',
 'the',
 'huge',
 'reversal',
 'in',
 'fee',
 'income',
 ',',
 'un',
 '##heard',
 'of',
 'among',
 'private',
 'sector',
 'lend',
 '##ers',
 '.',
 'essentially',
 ',',
 'it',
 'means',
 'that',
 'yes',
 'bank',
 'took',
 'it',
 'for',
 'granted',
 'that',
 'fees',
 'on',
 'structured',
 'loan',
 'deals',
 'will',
 'be',
 'paid',
 'and',
 'accounted',
 'for',
 'up',
 '##front',
 'on',
 'its',
 'books',
 '.',
 'as',
 'borrow',
 '##ers',
 'turned',
 'default',
 '##ers',
 ',',
 'the',
 'fees',
 'tied',
 'to',
 'these',
 'loan',
 'deals',
 'fell',
 'off',
 'the',
 'cracks',
 '.',
 'gill',
 'has',
 'now',
 'vowed',
 'to',
 'shift',
 'to',
 'a',
 'safer',
 'accounting',
 'practice',
 'of',
 'amor',
 '##ti',
 '##zing',
 'fee',
 'income',
 'rather',
 'than',
 'booking',
 'these',
 'up',
 '##front',
 '.',
 'gill',
 '’',
 's',
 'move',
 'to',
 'men',
 '##d',
 'past',
 'ways',
 'means',
 'that',
 'there',
 'will',
 'be',
 'no',
 'nasty',
 'sur